In [12]:
# Importing the necessary library's

import pandas as pd
import base64
import os
from requests import get, post
import json
import time
import numpy as np

pd.set_option("display.max_rows", 50)
pd.set_option("display.expand_frame_repr", True)
pd.set_option('display.width', 1000)

Read in chart data, then isolate to only United States

In [2]:
# Reading the dataset and creating a dataframe
raw_data = pd.read_csv('charts.csv',
                       parse_dates=['date'])
raw_data = raw_data[raw_data['region'] == "United States"]
print(raw_data.isna().sum())
#stream numbers are missing only

title          0
rank           0
date           0
artist         0
url            0
region         0
chart          0
trend          0
streams    90873
dtype: int64


1. sort dataframe on date
2. drop any songs that show up on the same date, chart.. there are 57 of these and we are only keeping the first occurence
3. create new column 'song_num_days' which is a count of how many days a song is on either the top 50 playlist or the top 200 playlist
4. create new column 'num_days_cumulative' which is a cumulative count of how many days a song has been on each chart so far
5. create new column 'predict_days_stay'chart' which is how many more days the song will stay on that chart

In [76]:
def formatartist(x):
  return x.strip().upper().strip('#')


raw_data.sort_values(by='date', inplace=True)
raw_data.drop_duplicates(subset=['chart','title', 'artist', 'date'], inplace=True, keep='first')
raw_data['artist'] = raw_data['artist'].apply(lambda x: formatartist(x))


raw_data['predict_days_stay'] = raw_data.groupby(['chart','title','artist'])['date'].transform('nunique')
raw_data['num_days_cumulative'] = raw_data.groupby(['chart','title','artist'])['date'].cumcount()
raw_data['predict_days_stay'] = raw_data.apply(lambda row: row['predict_days_stay'] - row['num_days_cumulative'], axis = 1)

raw_data['Num_artists'] = raw_data.apply(lambda row: len(row.artist.split(',')), axis = 1)

raw_data

,title,rank,date,artist,url,region,chart,trend,streams,predict_days_stay,num_days_cumulative,Num_artists
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,MIGOS,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,429,0,1
8371,You Can Call Me Al,125,2017-01-01,PAUL SIMON,https://open.spotify.com/track/0qxYx4F3vm1AOnf...,United States,top200,NEW_ENTRY,183669.0,1,0,1
8247,Fake Love,2,2017-01-01,DRAKE,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,455,0,1
8248,Starboy,3,2017-01-01,"THE WEEKND, DAFT PUNK",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,502,0,2
8249,Closer,4,2017-01-01,"THE CHAINSMOKERS, HALSEY",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,538,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
22692014,Whiskey Glasses,104,2021-12-31,MORGAN WALLEN,https://open.spotify.com/track/6foY66mWZN0pSRj...,United States,top200,MOVE_UP,263279.0,1,970,1
22692013,Drip Too Hard (Lil Baby & Gunna),103,2021-12-31,LIL BABY,https://open.spotify.com/track/78QR3Wp35dqAhFE...,United States,top200,MOVE_DOWN,263406.0,1,1107,1
22692040,Looking Out for You,130,2021-12-31,JOY AGAIN,https://open.spotify.com/track/3jfZ9M23l0L7Rxz...,United States,top200,MOVE_DOWN,237056.0,1,95,1
22692025,telepatía,115,2021-12-31,KALI UCHIS,https://open.spotify.com/track/6tDDoYIxWvMLTdK...,United States,top200,MOVE_UP,248962.0,1,312,1


Next we will merge in the track data to the final dataset
I am merging based on track [title, artist]

In [63]:
tracks = pd.read_csv('charts_US_attributes.csv')
track_columns = [ 'title', 
                 'url',
                 'artist', 
                 'danceability', 
                 'energy', 
                 'key', 
                 'loudness',
                 'mode', 
                 'speechiness', 
                 'acousticness', 
                 'instrumentalness', 
                 'liveness',
                 'valence', 
                 'tempo', 
                 'duration_ms', 
                 'time_signature']

tracks = tracks[track_columns]
tracks['artist'] = tracks['artist'].apply(lambda x: formatartist(x))
print(tracks.isna().sum())
tracks.drop_duplicates(subset=['url'], inplace=True)
print("track data we have: ",len(tracks))


title               0
url                 0
artist              0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
dtype: int64
track data we have:  12672


In [64]:
#print(raw_data.columns)
#print(tracks.columns)

final_data = pd.merge(raw_data, tracks, 
                      on=['url', 'title', 'artist'],
                      how='inner')
print("raw:   ",  len(raw_data))
print("final: " , len(final_data))  
print("diff:  " , len(raw_data) - len(final_data))  

print(final_data.columns)

final_data.sort_values(by='date', inplace=True)

print(final_data.isna().sum())
'''
missing = final_data[final_data.title_y.isna()]
missing = missing[['title_x', 'artist_x', 'url']]
missing.drop_duplicates(inplace= True)
missing.rename(columns={'title_x': 'title', 
                        'artist_x': 'artist', }, inplace=True )

missing.to_csv("missing.csv")
'''

raw:    455010
final:  390278
diff:   64732
Index(['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend', 'streams', 'predict_days_stay', 'num_days_cumulative', 'Num_artists', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'], dtype='object')
title                      0
rank                       0
date                       0
artist                     0
url                        0
region                     0
chart                      0
trend                      0
streams                80850
predict_days_stay          0
num_days_cumulative        0
Num_artists                0
danceability               0
energy                     0
key                        0
loudness                   0
mode                       0
speechiness                0
acousticness               0
instrumentalness           0
liveness                   0
valence       

'\nmissing = final_data[final_data.title_y.isna()]\nmissing = missing[[\'title_x\', \'artist_x\', \'url\']]\nmissing.drop_duplicates(inplace= True)\nmissing.rename(columns={\'title_x\': \'title\', \n                        \'artist_x\': \'artist\', }, inplace=True )\n\nmissing.to_csv("missing.csv")\n'

In [65]:
print(final_data.isna().sum())

title                      0
rank                       0
date                       0
artist                     0
url                        0
region                     0
chart                      0
trend                      0
streams                80850
predict_days_stay          0
num_days_cumulative        0
Num_artists                0
danceability               0
energy                     0
key                        0
loudness                   0
mode                       0
speechiness                0
acousticness               0
instrumentalness           0
liveness                   0
valence                    0
tempo                      0
duration_ms                0
time_signature             0
dtype: int64


next we will merge the artists data, queries from spotify, into the main dataset

In [77]:
artists_columns = [ 
                    'artist_id',
                    'artist_followers',
                    'genres',
                    'artist_popularity',
                    'artist_name',
                    'type',
                    'album_names',
                    'release_dates',
                    'artist_total_tracks'
                  ]
artists = pd.read_csv('artist_data.csv')
artists.rename(columns={'id': 'artist_id', 
                        'followers': 'artist_followers', 
                        'popularity': 'artist_popularity', 
                        'name': 'artist', 
                        'total_tracks':'artist_total_tracks'}, inplace=True )


artists['artist'] = artists['artist'].apply(lambda x: formatartist(x))
artists.drop_duplicates(subset='artist', inplace=True)
print(artists.columns)


Index(['artist_id', 'artist_followers', 'genres', 'artist_popularity', 'artist', 'type', 'album_names', 'release_dates', 'artist_total_tracks'], dtype='object')


In [75]:
print("total needed: ", len(final_data))

singleartists = final_data[final_data['Num_artists'] == 1]
singleartists['artist'] = singleartists['artist'].str.upper()
print("single needed: ", len(singleartists))

multipleartists = final_data[final_data['Num_artists'] > 1]
print("multiple needed: ", len(multipleartists))


final_single = pd.merge(singleartists, artists, 
                        on='artist', 
                        how='left')
print("single found: ", len(final_single))
print("single missing: ",  len(singleartists) - len(final_single))
final_single[final_single.artist_id.isna()]


missing = final_single[final_single.artist_id.isna()]
missing = missing[['artist']]
missing.drop_duplicates(inplace= True)

#missing.to_csv("missingartists.csv")
missing

total needed:  390278
single needed:  323287
multiple needed:  66991


C:\Users\aporfido\AppData\Local\Temp\ipykernel_34264\2328743707.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  singleartists['artist'] = singleartists['artist'].str.upper()


single found:  323287
single missing:  0


,artist
35,MANWOLVES
63,VICE
109,KYLE
118,SHELLEY FKA DRAM
168,YEVIN FRANCISCO
...,...
321134,ETTA JAMES
321790,JMO DIVINCI
321795,RAP PLUG
322736,GOLDSKULL
